In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from past.builtins import xrange
import datetime
from datetime import timedelta
from pandas import DataFrame
from scipy.fftpack import fft, ifft,dct
import pywt
import scipy.stats as stats
from sklearn.metrics import classification_report, accuracy_score, make_scorer

import warnings
warnings.filterwarnings("ignore")

In [2]:
insulin_data1=pd.read_csv('InsulinData.csv',low_memory=False, parse_dates = [['Date', 'Time']])[['Date_Time', 'BWZ Estimate (U)']]
cgm_data1=pd.read_csv('CGMData.csv',low_memory=False,parse_dates = [['Date', 'Time']])

In [3]:
def trimData(insulin_data, cgm_data):
    min_insulin_time = min(insulin_data['Date_Time'])
    min_cgm_time = min(cgm_data['Date_Time'])
    min_timestamp = max(min_insulin_time, min_cgm_time)
    max_insulin_time = max(insulin_data['Date_Time'])
    max_cgm_time = max(cgm_data['Date_Time'])
    max_timestamp = min(max_insulin_time, max_cgm_time)
    insulin_trimmed = insulin_data[(insulin_data['Date_Time'] >= min_timestamp)]
    insulin_trimmed = insulin_trimmed.reset_index(drop=True)
    insulin_trimmed = insulin_data[(insulin_data['Date_Time'] <= max_timestamp)]
    insulin_trimmed = insulin_trimmed.reset_index(drop=True)
    cgm_trimmed = cgm_data[(cgm_data['Date_Time'] >= min_timestamp)]
    cgm_trimmed = cgm_trimmed.reset_index(drop=True)
    cgm_trimmed = cgm_data[(cgm_data['Date_Time'] <= max_timestamp)]
    cgm_trimmed = cgm_trimmed.reset_index(drop=True)
    return insulin_trimmed, cgm_trimmed

In [4]:
insulin_data1_trimmed, cgm_data1_trimmed = trimData(insulin_data1, cgm_data1)

In [5]:
Bool_carbinput_data1=pd.notnull(insulin_data1_trimmed['BWZ Estimate (U)'])
insuline_data_ext1=insulin_data1_trimmed[Bool_carbinput_data1]
insuline_data_ext1 = insuline_data_ext1[insuline_data_ext1['BWZ Estimate (U)'] != 0].sort_values(by=['Date_Time'], ignore_index = True)
insuline_data_ext1 = insuline_data_ext1.reset_index(drop=True)

In [6]:
def extract_meal_cgm_timestamps(meal_dataframe):
    meal_data = []
    time_val = []
    meal_len = len(meal_dataframe)-1
    for row in range(0,meal_len):
        new_time = meal_dataframe.at[row, 'Date_Time'] + timedelta(hours = 2)
        if(new_time > meal_dataframe.at[row+1, 'Date_Time']):
           continue
        else:
             meal_data.append(meal_dataframe.at[row, 'Date_Time'])
             time_val.append(meal_dataframe.at[row, 'BWZ Estimate (U)'],)
    meal_dataframe_fin = pd.DataFrame(meal_data)
    meal_dataframe_fin['val'] = time_val
    return meal_dataframe_fin      

In [7]:
# result = []
meal_data1 = extract_meal_cgm_timestamps(insuline_data_ext1)
print(meal_data1)

                      0   val
0   2017-07-25 10:39:46   6.4
1   2017-07-25 18:31:40  12.1
2   2017-07-26 12:48:41   7.4
3   2017-07-26 19:15:06   6.2
4   2017-07-27 05:45:51   2.8
..                  ...   ...
718 2018-02-11 12:43:23   3.2
719 2018-02-11 18:14:37   1.0
720 2018-02-11 20:33:18   8.8
721 2018-02-12 00:15:41   2.5
722 2018-02-12 02:30:55   1.2

[723 rows x 2 columns]


In [8]:
cgm_data1_trimmed['Sensor Glucose (mg/dL)'] = cgm_data1_trimmed['Sensor Glucose (mg/dL)'].interpolate(method = 'linear')

In [9]:
cgm_data1_trimmed = cgm_data1_trimmed[['Date_Time','Sensor Glucose (mg/dL)']]
cgm_data1_trimmed = cgm_data1_trimmed.reindex(index=cgm_data1_trimmed.index[::-1])
cgm_data1_trimmed = cgm_data1_trimmed.reset_index(drop=True)

In [10]:
def meal_cgm_extraction(meal_cgm_dataframe, cgmdata):
    list1 = ['cgm_val'+str(x) for x in range(30)]
    meal_data = pd.DataFrame(columns = list1)
    next_hit_list = []
    for id in meal_cgm_dataframe.index:
        dict1 = dict()
        data_sets = cgmdata[cgmdata['Date_Time'] >= meal_cgm_dataframe[0][id]]
        data_set_list = list(cgmdata.loc[data_sets.index[0]-6: data_sets.index[0]+23, 'Sensor Glucose (mg/dL)'].values)
        next_hit_list.append(cgmdata.loc[data_sets.index[0], 'Sensor Glucose (mg/dL)'])
        cgm_list=[]
        for id1, cgm_val in enumerate(data_set_list):
            cgm_list.append(cgm_val)
            for cgm in cgm_list:
              dict1[list1[id1]] = cgm_val
        meal_data = meal_data.append(dict1, ignore_index = True)
    meal_data['val']=list(meal_cgm_dataframe['val'])
    return meal_data,next_hit_list

In [11]:
cgm_meal1,next_hit_list = meal_cgm_extraction(meal_data1, cgm_data1_trimmed)
meal_cgm_master = cgm_meal1.reset_index(drop=True)
meal_cgm_master_list = meal_cgm_master.values.tolist()

In [12]:
meal_list_fin = []
meal_df = meal_cgm_master.drop(columns=['val'])
print(meal_df)
meal_list = meal_df.values.tolist()
for row in meal_list:
  for i in row:
    meal_list_fin.append(i)
max_val = max(meal_list_fin) 
min_val = min(meal_list_fin)
print(min_val)
print(max_val,min_val)
bin_size=int(max_val)//20
bins=range(bin_size)
dict1={}
for i in range(0,len(bins)-1):
  if i ==0:
    dict1[i]=min_val
  else:
    dict1[i]=dict1[i-1]+20
dict1.pop(0)
print(dict1)

       cgm_val0    cgm_val1    cgm_val2  ...  cgm_val27  cgm_val28  cgm_val29
0    314.000000  310.000000  309.000000  ...        NaN        NaN        NaN
1     58.000000   59.000000   63.000000  ...      198.0      203.0      196.0
2    304.000000  292.000000  281.000000  ...      284.0      283.0      278.0
3     40.000000   40.000000   40.000000  ...       74.0       77.0       81.0
4    212.000000  210.000000  204.000000  ...      209.0      210.0      209.0
..          ...         ...         ...  ...        ...        ...        ...
718  209.000000  219.000000  221.000000  ...      212.0      215.0      208.0
719  169.176471  167.941176  166.705882  ...      123.0      106.0      107.0
720  106.000000  107.000000  100.000000  ...      200.0      194.0      177.0
721  250.000000  245.000000  250.000000  ...       89.0       80.0       69.0
722   89.000000   80.000000   69.000000  ...       92.0       93.0       86.0

[723 rows x 30 columns]
40.0
400.0 40.0
{1: 60.0, 2: 80.0, 3: 1

In [13]:
print(dict1)
b_max = meal_df.max(axis=1)
b_max_list = b_max.values.tolist()
print(b_max_list)
b_max_binlist=[]
list1 = list(dict1.values())
print(list1)
for i in b_max_list:
  if i<=list1[0]:
    b_max_binlist.append(1)
  elif i<=list1[1]:
    b_max_binlist.append(2)
  elif i<=list1[2]:
    b_max_binlist.append(3)
  elif i<=list1[3]:
    b_max_binlist.append(4)
  elif i<=list1[4]:
    b_max_binlist.append(5)
  elif i<=list1[5]:
    b_max_binlist.append(6)
  elif i<=list1[6]:
    b_max_binlist.append(7)
  elif i<=list1[7]:
    b_max_binlist.append(8)
  elif i<=list1[8]:
    b_max_binlist.append(9)
  elif i<=list1[9]:
    b_max_binlist.append(10)
  elif i<=list1[10]:
    b_max_binlist.append(11)
  elif i<=list1[11]:
    b_max_binlist.append(12)
  elif i<=list1[12]:
    b_max_binlist.append(13)
  elif i<=list1[13]:
    b_max_binlist.append(14)
  elif i<=list1[14]:
    b_max_binlist.append(15)
  elif i<=list1[15]:
    b_max_binlist.append(16)
  elif i<=list1[16]:
    b_max_binlist.append(17)
  elif i<=list1[17]:
    b_max_binlist.append(18)
print(len(b_max_binlist))    
meal_cgm_master['B_max']=b_max_binlist 

{1: 60.0, 2: 80.0, 3: 100.0, 4: 120.0, 5: 140.0, 6: 160.0, 7: 180.0, 8: 200.0, 9: 220.0, 10: 240.0, 11: 260.0, 12: 280.0, 13: 300.0, 14: 320.0, 15: 340.0, 16: 360.0, 17: 380.0, 18: 400.0}
[324.0, 203.0, 304.0, 132.0, 216.0, 213.0, 200.0, 226.0, 175.0, 272.0, 251.0, 134.0, 201.0, 277.0, 245.0, 276.0, 134.6, 105.19999999999999, 87.2, 88.0, 181.0, 267.0, 207.0, 224.0, 226.0, 192.0, 157.0, 248.0, 247.0, 160.0, 181.0, 273.0, 144.0, 251.0, 268.0, 380.0, 272.0, 312.0, 302.0, 255.4022988505747, 244.3448275862069, 236.57471264367817, 268.0, 263.0, 181.0, 290.0, 267.0, 318.0, 194.0, 287.0, 243.0, 226.0, 335.0, 312.0, 88.0, 286.0, 206.0, 211.0, 152.0, 161.0, 221.0, 238.0, 239.0, 253.0, 313.0, 156.0, 285.0, 151.0, 141.0, 163.0, 251.0, 362.0, 135.0, 167.0, 197.0, 228.0, 132.0, 156.0, 154.0, 185.0, 180.0, 117.0, 184.0, 124.0, 180.0, 321.0, 219.0, 190.0, 199.0, 232.0, 259.0, 196.0, 377.0, 162.0, 270.0, 383.0, 355.0, 357.0, 366.0, 400.0, 236.0, 167.0, 139.0, 124.35616438356163, 298.32876712328766, 354

In [14]:
b_min_binlist = []
for j in next_hit_list:
  i=int(j)
  if i<=list1[0]:
    b_min_binlist.append(1)
  elif i<=list1[1]:
    b_min_binlist.append(2)
  elif i<=list1[2]:
    b_min_binlist.append(3)
  elif i<=list1[3]:
    b_min_binlist.append(4)
  elif i<=list1[4]:
    b_min_binlist.append(5)
  elif i<=list1[5]:
    b_min_binlist.append(6)
  elif i<=list1[6]:
    b_min_binlist.append(7)
  elif i<=list1[7]:
    b_min_binlist.append(8)
  elif i<=list1[8]:
    b_min_binlist.append(9)
  elif i<=list1[9]:
    b_min_binlist.append(10)
  elif i<=list1[10]:
    b_min_binlist.append(11)
  elif i<=list1[11]:
    b_min_binlist.append(12)
  elif i<=list1[12]:
    b_min_binlist.append(13)
  elif i<=list1[13]:
    b_min_binlist.append(14)
  elif i<=list1[14]:
    b_min_binlist.append(15)
  elif i<=list1[15]:
    b_min_binlist.append(16)
  elif i<=list1[16]:
    b_min_binlist.append(17)
  elif i<=list1[17]:
    b_min_binlist.append(18)
print(len(b_min_binlist),b_min_binlist)    

723 [14, 5, 11, 3, 9, 5, 1, 8, 7, 6, 9, 2, 8, 8, 9, 11, 5, 4, 3, 2, 4, 11, 7, 4, 7, 7, 6, 6, 4, 3, 4, 9, 3, 5, 11, 17, 10, 8, 5, 11, 11, 10, 7, 10, 3, 4, 9, 14, 8, 12, 9, 9, 5, 13, 2, 11, 5, 8, 6, 6, 4, 6, 10, 5, 14, 2, 7, 5, 2, 6, 6, 15, 1, 6, 5, 2, 4, 4, 6, 5, 5, 4, 6, 1, 7, 9, 3, 5, 8, 5, 11, 4, 15, 2, 5, 16, 1, 14, 10, 18, 7, 7, 4, 5, 9, 15, 12, 4, 3, 8, 5, 3, 9, 9, 12, 4, 5, 11, 8, 9, 6, 15, 7, 13, 4, 8, 9, 4, 6, 10, 6, 15, 4, 4, 2, 6, 4, 7, 12, 8, 7, 3, 11, 2, 4, 4, 11, 3, 15, 15, 12, 5, 7, 9, 5, 8, 5, 5, 12, 4, 5, 7, 9, 6, 4, 6, 5, 4, 8, 2, 6, 4, 11, 5, 5, 3, 6, 5, 7, 12, 5, 7, 9, 7, 9, 6, 7, 5, 5, 9, 7, 4, 12, 2, 7, 4, 7, 7, 8, 5, 4, 10, 7, 4, 5, 4, 6, 12, 12, 6, 5, 9, 10, 5, 13, 8, 4, 4, 6, 4, 3, 11, 12, 8, 11, 7, 1, 11, 6, 1, 8, 9, 7, 5, 7, 8, 9, 11, 4, 3, 8, 9, 14, 6, 6, 4, 12, 6, 6, 11, 8, 12, 10, 6, 7, 4, 3, 2, 10, 4, 12, 2, 7, 9, 6, 5, 13, 1, 4, 2, 2, 10, 5, 5, 8, 6, 6, 13, 10, 3, 17, 9, 6, 3, 8, 7, 5, 8, 5, 15, 6, 12, 10, 7, 4, 5, 5, 10, 4, 3, 7, 3, 4, 6, 5, 12, 12, 7, 1

In [15]:
meal_cgm_master['B_max']=b_max_binlist 
meal_cgm_master['B_min']=b_min_binlist 

In [16]:
meal_cgm_master

,cgm_val0,cgm_val1,cgm_val2,cgm_val3,cgm_val4,cgm_val5,cgm_val6,cgm_val7,cgm_val8,cgm_val9,cgm_val10,cgm_val11,cgm_val12,cgm_val13,cgm_val14,cgm_val15,cgm_val16,cgm_val17,cgm_val18,cgm_val19,cgm_val20,cgm_val21,cgm_val22,cgm_val23,cgm_val24,cgm_val25,cgm_val26,cgm_val27,cgm_val28,cgm_val29,val,B_max,B_min
0,314.000000,310.000000,309.000000,311.000000,311.000000,311.0,312.000000,312.000000,316.000000,324.000000,319.000000,315.000000,314.000000,316.000000,314.000000,310.000000,298.000000,293.000000,279.000000,278.000000,281.000000,289.000000,294.0,290.0,NaN,NaN,NaN,NaN,NaN,NaN,6.4,15,14
1,58.000000,59.000000,63.000000,71.000000,81.000000,102.0,131.000000,140.000000,147.000000,153.000000,159.000000,163.000000,159.000000,153.000000,151.000000,155.000000,163.000000,164.000000,165.000000,166.000000,168.000000,164.000000,169.0,178.0,184.0,190.0,195.0,198.0,203.0,196.0,12.1,9,5
2,304.000000,292.000000,281.000000,268.000000,259.000000,255.0,248.000000,241.000000,231.000000,220.000000,222.000000,222.000000,222.000000,223.000000,224.000000,228.000000,233.000000,243.000000,256.000000,262.000000,270.000000,277.000000,274.0,269.0,267.0,267.0,274.0,284.0,283.0,278.0,7.4,14,11
3,40.000000,40.000000,40.000000,40.000000,60.000000,71.0,83.000000,87.000000,100.000000,112.000000,121.000000,125.000000,130.000000,132.000000,123.000000,112.000000,89.000000,77.000000,69.000000,66.000000,67.000000,71.000000,75.0,74.0,72.0,70.0,67.0,74.0,77.0,81.0,6.2,5,3
4,212.000000,210.000000,204.000000,200.000000,199.000000,201.0,201.000000,194.000000,188.000000,183.000000,181.000000,176.000000,170.000000,168.000000,169.000000,172.000000,179.000000,192.000000,203.000000,205.000000,210.000000,213.000000,212.0,216.0,213.0,210.0,210.0,209.0,210.0,209.0,2.8,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,209.000000,219.000000,221.000000,221.000000,216.000000,217.0,215.000000,211.000000,201.000000,200.000000,198.000000,196.000000,194.000000,200.000000,220.000000,207.000000,205.000000,194.000000,186.000000,177.000000,180.000000,184.000000,180.0,190.0,196.0,203.0,206.0,212.0,215.0,208.0,3.2,10,9
719,169.176471,167.941176,166.705882,165.470588,164.235294,163.0,161.764706,160.529412,159.294118,158.058824,156.823529,155.588235,154.352941,153.117647,151.882353,150.647059,149.411765,148.176471,146.941176,145.705882,144.470588,143.235294,142.0,145.0,147.0,137.0,128.0,123.0,106.0,107.0,1.0,7,7
720,106.000000,107.000000,100.000000,104.000000,123.000000,132.0,129.000000,127.000000,120.000000,132.000000,138.000000,145.000000,160.000000,171.000000,178.000000,182.000000,180.000000,168.000000,166.000000,167.000000,169.000000,176.000000,173.0,166.0,162.0,189.0,201.0,200.0,194.0,177.0,8.8,9,5
721,250.000000,245.000000,250.000000,259.000000,262.000000,260.0,279.000000,289.000000,288.000000,287.000000,284.000000,277.000000,263.000000,242.000000,228.000000,201.000000,191.000000,186.000000,185.000000,186.000000,178.000000,160.000000,139.0,123.0,111.0,100.0,98.0,89.0,80.0,69.0,2.5,13,12


In [17]:
import math
def my_round(i):
  f = math.floor(i)
  return f if i - f < 0.5 else f+1
meal_cgm_master['val'] = [my_round(i) for i in meal_cgm_master['val']]

In [18]:
meal_cgm_master

,cgm_val0,cgm_val1,cgm_val2,cgm_val3,cgm_val4,cgm_val5,cgm_val6,cgm_val7,cgm_val8,cgm_val9,cgm_val10,cgm_val11,cgm_val12,cgm_val13,cgm_val14,cgm_val15,cgm_val16,cgm_val17,cgm_val18,cgm_val19,cgm_val20,cgm_val21,cgm_val22,cgm_val23,cgm_val24,cgm_val25,cgm_val26,cgm_val27,cgm_val28,cgm_val29,val,B_max,B_min
0,314.000000,310.000000,309.000000,311.000000,311.000000,311.0,312.000000,312.000000,316.000000,324.000000,319.000000,315.000000,314.000000,316.000000,314.000000,310.000000,298.000000,293.000000,279.000000,278.000000,281.000000,289.000000,294.0,290.0,NaN,NaN,NaN,NaN,NaN,NaN,6,15,14
1,58.000000,59.000000,63.000000,71.000000,81.000000,102.0,131.000000,140.000000,147.000000,153.000000,159.000000,163.000000,159.000000,153.000000,151.000000,155.000000,163.000000,164.000000,165.000000,166.000000,168.000000,164.000000,169.0,178.0,184.0,190.0,195.0,198.0,203.0,196.0,12,9,5
2,304.000000,292.000000,281.000000,268.000000,259.000000,255.0,248.000000,241.000000,231.000000,220.000000,222.000000,222.000000,222.000000,223.000000,224.000000,228.000000,233.000000,243.000000,256.000000,262.000000,270.000000,277.000000,274.0,269.0,267.0,267.0,274.0,284.0,283.0,278.0,7,14,11
3,40.000000,40.000000,40.000000,40.000000,60.000000,71.0,83.000000,87.000000,100.000000,112.000000,121.000000,125.000000,130.000000,132.000000,123.000000,112.000000,89.000000,77.000000,69.000000,66.000000,67.000000,71.000000,75.0,74.0,72.0,70.0,67.0,74.0,77.0,81.0,6,5,3
4,212.000000,210.000000,204.000000,200.000000,199.000000,201.0,201.000000,194.000000,188.000000,183.000000,181.000000,176.000000,170.000000,168.000000,169.000000,172.000000,179.000000,192.000000,203.000000,205.000000,210.000000,213.000000,212.0,216.0,213.0,210.0,210.0,209.0,210.0,209.0,3,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,209.000000,219.000000,221.000000,221.000000,216.000000,217.0,215.000000,211.000000,201.000000,200.000000,198.000000,196.000000,194.000000,200.000000,220.000000,207.000000,205.000000,194.000000,186.000000,177.000000,180.000000,184.000000,180.0,190.0,196.0,203.0,206.0,212.0,215.0,208.0,3,10,9
719,169.176471,167.941176,166.705882,165.470588,164.235294,163.0,161.764706,160.529412,159.294118,158.058824,156.823529,155.588235,154.352941,153.117647,151.882353,150.647059,149.411765,148.176471,146.941176,145.705882,144.470588,143.235294,142.0,145.0,147.0,137.0,128.0,123.0,106.0,107.0,1,7,7
720,106.000000,107.000000,100.000000,104.000000,123.000000,132.0,129.000000,127.000000,120.000000,132.000000,138.000000,145.000000,160.000000,171.000000,178.000000,182.000000,180.000000,168.000000,166.000000,167.000000,169.000000,176.000000,173.0,166.0,162.0,189.0,201.0,200.0,194.0,177.0,9,9,5
721,250.000000,245.000000,250.000000,259.000000,262.000000,260.0,279.000000,289.000000,288.000000,287.000000,284.000000,277.000000,263.000000,242.000000,228.000000,201.000000,191.000000,186.000000,185.000000,186.000000,178.000000,160.000000,139.0,123.0,111.0,100.0,98.0,89.0,80.0,69.0,3,13,12


In [19]:
import collections
dict_bs = collections.Counter([(i,j) for i,j in zip(meal_cgm_master['B_min'],meal_cgm_master['B_max'])])
print(dict_bs)

Counter({(6, 7): 25, (12, 12): 24, (7, 8): 21, (6, 8): 20, (11, 12): 19, (9, 10): 19, (5, 9): 18, (8, 8): 18, (8, 9): 17, (10, 10): 17, (12, 13): 17, (4, 6): 17, (7, 7): 15, (7, 9): 15, (6, 6): 14, (11, 11): 14, (4, 9): 14, (3, 7): 14, (9, 11): 13, (5, 6): 13, (4, 7): 13, (3, 6): 12, (5, 8): 12, (5, 10): 12, (8, 10): 11, (5, 5): 11, (4, 8): 11, (7, 10): 11, (3, 8): 11, (9, 9): 10, (10, 11): 10, (6, 10): 9, (5, 7): 9, (3, 5): 8, (4, 11): 8, (9, 12): 8, (13, 14): 8, (4, 10): 7, (6, 11): 7, (2, 3): 6, (5, 11): 6, (6, 9): 6, (8, 11): 6, (3, 4): 6, (2, 5): 5, (10, 12): 5, (11, 13): 5, (3, 9): 5, (2, 7): 5, (15, 16): 5, (6, 13): 5, (2, 4): 5, (11, 14): 4, (6, 12): 4, (14, 14): 4, (2, 6): 4, (7, 13): 4, (1, 5): 4, (13, 15): 4, (3, 10): 4, (14, 15): 3, (8, 12): 3, (4, 4): 3, (5, 14): 3, (4, 13): 3, (15, 17): 3, (4, 5): 3, (5, 12): 3, (9, 13): 3, (12, 15): 3, (6, 15): 3, (16, 17): 3, (1, 8): 2, (3, 3): 2, (17, 17): 2, (7, 12): 2, (5, 15): 2, (9, 15): 2, (16, 18): 2, (14, 16): 2, (2, 9): 2, (7, 

In [20]:
dict_bs2 = collections.Counter([(i,j,k) for i,j,k in zip(meal_cgm_master['B_min'],meal_cgm_master['B_max'],meal_cgm_master['val'])])
dict_bs2

Counter({(1, 3, 1): 1,
         (1, 4, 8): 1,
         (1, 5, 1): 1,
         (1, 5, 3): 1,
         (1, 5, 4): 1,
         (1, 5, 9): 1,
         (1, 8, 2): 1,
         (1, 8, 6): 1,
         (1, 9, 9): 1,
         (1, 11, 5): 1,
         (1, 12, 3): 1,
         (1, 16, 2): 1,
         (2, 2, 6): 1,
         (2, 2, 9): 1,
         (2, 3, 5): 1,
         (2, 3, 6): 3,
         (2, 3, 8): 1,
         (2, 3, 12): 1,
         (2, 4, 0): 1,
         (2, 4, 1): 1,
         (2, 4, 3): 1,
         (2, 4, 7): 2,
         (2, 5, 2): 1,
         (2, 5, 3): 1,
         (2, 5, 4): 1,
         (2, 5, 6): 1,
         (2, 5, 8): 1,
         (2, 6, 1): 1,
         (2, 6, 4): 1,
         (2, 6, 5): 1,
         (2, 6, 9): 1,
         (2, 7, 1): 2,
         (2, 7, 3): 1,
         (2, 7, 6): 1,
         (2, 7, 7): 1,
         (2, 8, 2): 1,
         (2, 9, 1): 1,
         (2, 9, 7): 1,
         (2, 10, 7): 1,
         (3, 3, 1): 1,
         (3, 3, 2): 1,
         (3, 4, 1): 2,
         (3, 4, 2): 1,
      

In [43]:
dict_bs3 = {}
for i in dict_bs2.keys():
  dict_bs3[i] = dict_bs2[i]/dict_bs[(i[0],i[1])]
new_sorted = sorted(dict_bs3,key=lambda x:dict_bs3[x],reverse=True)#last csv
Largest_confidence_rules = []
Most_frequent_setlist = []
for i in new_sorted:
  if dict_bs3[i]==1.0:
    Largest_confidence_rules.append(i)
new_sorted2 = sorted(dict_bs2,key=lambda x:dict_bs2[x],reverse=True)#most frequent
for i in new_sorted2:
    if dict_bs2[i]==9:
      Most_frequent_setlist.append(i)
print(Largest_confidence_rules) 
print(Most_frequent_setlist)
#print(new_sorted)
#print(new_sorted2)

[(8, 14, 4), (2, 10, 7), (9, 15, 4), (1, 16, 2), (10, 17, 8), (18, 18, 3), (8, 18, 10), (9, 14, 6), (15, 18, 11), (4, 14, 12), (1, 11, 5), (1, 12, 3), (6, 14, 4), (1, 4, 8), (1, 3, 1), (12, 14, 1), (8, 13, 10), (5, 13, 6), (13, 17, 2), (2, 8, 2), (1, 9, 9), (11, 15, 7)]
[(7, 8, 1), (8, 8, 1), (8, 9, 1), (6, 7, 1)]


In [44]:
#CSV With Most Frequent Sets
df_mostfrequent = DataFrame()
df_mostfrequent['Most Frequent Rules/Sets'] = Most_frequent_setlist
df_largestConfidence = DataFrame()
df_largestConfidence['Largest Confidence Rules/Sets'] = Largest_confidence_rules

In [45]:
df_mostfrequent.to_csv('Most Frequent Sets.csv', index=False)
df_largestConfidence.to_csv('Largest Confidence Sets.csv', index=False)

In [46]:
count=[]
for i in  new_sorted:
  if(dict_bs3[i]<0.15):
    count.append(i)
sorted_sets = sorted(count)
print(sorted_sets)   

[(3, 5, 2), (3, 5, 4), (3, 5, 12), (3, 6, 2), (3, 6, 4), (3, 6, 5), (3, 6, 7), (3, 6, 8), (3, 6, 9), (3, 6, 10), (3, 6, 12), (3, 7, 1), (3, 7, 2), (3, 7, 4), (3, 7, 5), (3, 7, 6), (3, 8, 1), (3, 8, 3), (4, 6, 1), (4, 6, 3), (4, 6, 4), (4, 6, 5), (4, 6, 6), (4, 6, 9), (4, 6, 11), (4, 7, 5), (4, 7, 12), (4, 8, 4), (4, 8, 7), (4, 8, 9), (4, 8, 10), (4, 8, 13), (4, 9, 2), (4, 9, 5), (4, 9, 8), (4, 10, 3), (4, 10, 5), (4, 10, 13), (4, 11, 2), (4, 11, 6), (5, 5, 0), (5, 5, 4), (5, 5, 5), (5, 5, 7), (5, 5, 8), (5, 5, 9), (5, 6, 0), (5, 6, 3), (5, 6, 5), (5, 6, 6), (5, 6, 9), (5, 7, 2), (5, 7, 5), (5, 7, 11), (5, 8, 1), (5, 8, 6), (5, 8, 9), (5, 9, 1), (5, 9, 3), (5, 9, 5), (5, 9, 6), (5, 9, 9), (5, 9, 10), (5, 9, 11), (5, 9, 12), (5, 10, 1), (5, 10, 3), (5, 10, 6), (5, 10, 8), (5, 10, 10), (6, 6, 1), (6, 6, 3), (6, 6, 4), (6, 6, 7), (6, 6, 8), (6, 7, 3), (6, 7, 4), (6, 7, 6), (6, 7, 7), (6, 7, 9), (6, 7, 11), (6, 8, 5), (6, 8, 6), (6, 8, 12), (6, 10, 1), (6, 10, 8), (6, 11, 2), (6, 11, 3), (6

In [47]:
df_anomolousRules = DataFrame()
df_anomolousRules['Rules with Confidence < 15 %'] = sorted_sets

In [48]:
df_anomolousRules.to_csv('Confidence < 15% Sets.csv', index=False)